In [7]:
# coding:utf-8
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import random
import chainer
from chainer import Function, Variable, optimizers, serializers
from chainer import Link, Chain
import chainer.functions as F
import chainer.links as L
import copy
import gym

NUM_IN   = 2
NUM_HID1 = 1000
NUM_HID2 = 500
NUM_HID3 = 250
NUM_OUT  = 3
BATCH_SIZE = 100
ACTION = [-2, 0, 2]
EPOCH = 100
GAMMA = 0.1
EPSIL = 0.1
DQNMODEL_PATH = "./DQN_model/mounaincar.model"

def deep_q_learn(env, q_network, target_network, experince):
    # プロッセサーの選択
    xp = switch_ProccerUnit("CPU")
    epsil = EPSIL
    record = []
    for episode in range(1, EPOCH+1):
        print(episode)
        # 初期状態の定義
        now_state = env.reset()
        pos, spe = get_state(now_state)
        state_vec = get_state_vec(pos, spe, xp, 2)
        epsil = reduce_epsil(episode, epsil)
        for step in range(10000):
            # 方策に従って行動を選択
            action, _ = policy_egreedy_tri(state_vec, q_network, epsil, xp)
            # 行動による次状態を観測
            next_state, reward, terminal, info = agent_action(env, action)
            next_pos, next_spe = get_state(next_state)
            deep_learn(action, q_network, state_vec, xp, [make_target(action, reward, next_state, terminal, target_network, xp)])
            if reward == 1:
                print("成功")
                stock_record(pos, spe, action, 0, 0, reward, 1)
                break
            # 状態更新
            next_pos, next_spe = get_state(next_state)
            state_vec = get_state_vec(next_pos, next_spe, xp, 2)
            # レコード蓄積
            record = stock_record(record, pos, spe, action, next_pos, next_state, reward, 0)
            pos, spe = next_pos, next_spe
            # env.render()
            # Experience_Replayによるバッチ学習
            if len(record) > 1000:
                experience_replay(record, q_network, target_network, xp)
                if len(record) > 1500:
                    record = []
            # Target_networkの更新
            if step % 200 == 0:
                q_network.save_weight(DQNMODEL_PATH)
                target_network = update_target_network(q_network)
        
def get_state(state):
    return state[0], state[1]

def get_state_vec(pos, spe, xp, flg):
    vec = xp.array([pos, spe], dtype=xp.float32)
    if flg == 1: return vec
    return xp.array([vec], dtype=xp.float32) 
    
    
def experience_replay(record, q_network, target_network, xp):
    state_vecs, actions, next_states, rewards, terminals = transelate(record, xp)
    perm = xp.random.permutation(len(record))[:BATCH_SIZE]
    x_batch_state_vecs = state_vecs[perm[0:BATCH_SIZE]]
    x_batch_action     = actions[perm[0:BATCH_SIZE]]
    x_batch_rewards    = rewards[perm[0:BATCH_SIZE]]
    x_batch_terminals  = terminals[perm[0:BATCH_SIZE]]
    y_batch_targets    = []
    for index in range(BATCH_SIZE):
        y_batch_targets.append(make_target(x_batch_action[index], x_batch_rewards[index], next_states[perm[index]], x_batch_terminals[index], target_network, xp))
    deep_learn(x_batch_action, q_network, x_batch_state_vecs, xp, y_batch_targets)

def transelate(record, xp):
    state_vecs  = []
    actions     = []
    rewards     = []
    terminals   = []
    next_states = []
    for data in record:
        state_vecs.append(get_state_vec(data[0][0],data[0][1], xp, 1))
        actions.append(data[1])
        next_states.append(data[2])
        rewards.append(data[3])
        terminals.append(data[4])
    return xp.array(state_vecs, dtype=xp.float32), xp.array(actions), next_states, xp.array(rewards), xp.array(terminals)   
          
    
def stock_record(record, pos, spe, action, next_pos, next_spe, reward, terminal):
    record.append([(pos, spe), action, (next_pos, next_pos), reward, terminal])
    return record
    
    
def make_target(action, reward, next_state, tereminal, target_network, xp):
    y_target = [0 for i in range(3)]
    _, max_q = policy_egreedy_tri(get_state_vec(next_state[0], next_state[1], xp, 2), target_network, 0, xp)
    y_target[action] = reward if tereminal else reward + GAMMA * max_q    
    y_target = xp.array(y_target, dtype=np.float32)
    return y_target

def deep_learn(action, q_network, state_vec, xp, y_target, flg=None):
    y_target = xp.array(y_target, dtype=xp.float32)
    q_network.init_grads()
    loss = q_network.forward(1, state_vec, y_target)
    q_network.backpropagation(loss)
    
    
def policy_egreedy_tri(state, neural, epsil, xp):
    import scipy.spatial.distance
    qvalue_list = []
    tmp = []
    qvalue_list.append(neural.forward(0, state).data[0])
    qvalue_list.append(neural.forward(0, state).data[0])
    qvalue_vec = np.array(neural.forward(0, state).data[0])
    for qvalue in qvalue_list:
        sim = 1 - scipy.spatial.distance.cosine(xp.array(qvalue), qvalue_vec)
        tmp.append(sim)
    if tmp[0] < tmp[1]:
        return (list(qvalue_list[1]).index(max(qvalue_list[1])) if random.random()>epsil else random.choice([0,1,2])), max(qvalue_list[1]) 
    else:
        return (list(qvalue_list[0]).index(max(qvalue_list[0])) if random.random()>epsil else random.choice([0,1,2])), max(qvalue_list[0])
        
        
        
def agent_action(env, action):
    return env.step(action)

def update_target_network(q_network):
    return copy.deepcopy(q_network)

def reduce_epsil(epoch, epsil):
    return epsil
                 
    
def switch_ProccerUnit(pu):
    return cuda.cupy if pu == "GPU" else np
                 
    
def main():
    env = gym.make("MountainCar-v0")
    q_network = NeuralNetwork(NUM_IN, NUM_HID1, NUM_HID2, NUM_HID3, NUM_OUT)
    target_network = NeuralNetwork(NUM_IN, NUM_HID1, NUM_HID2, NUM_HID3, NUM_OUT)
    q_network.load_weight(DQNMODEL_PATH)
    target_network = update_target_network(q_network)
    deep_q_learn(env, q_network, target_network, [0])
        

In [6]:
# coding:utf-8
import chainer
from chainer import Function, Variable, optimizers, serializers
from chainer import Link, Chain
import chainer.functions as F
import chainer.links as L
import matplotlib.pyplot as plt
import numpy as np

class NeuralNetwork:
    def __init__(self, num_in, num_hid1, num_hid2, num_hid3, num_out):
        self.model = Chain(hid_layer1 = L.Linear(num_in, num_hid1),
                           hid_layer2 = L.Linear(num_hid1, num_hid2),
                           hid_layer3 = L.Linear(num_hid2, num_hid3),
                           out_layer  = L.Linear(num_hid3, num_out))
        self.optimizer = optimizers.Adam()
        self.optimizer.setup(self.model)
    
    def forward(self, flg, x, t = None):
        _x = Variable(x)
        if flg == 1: _t = Variable(t)
        h1  = F.dropout(F.relu(self.model.hid_layer1(_x)))
        h2  = F.dropout(F.relu(self.model.hid_layer2(h1)))
        h3  = F.dropout(F.relu(self.model.hid_layer3(h2)))
        u3  = self.model.out_layer(h3)
        # return F.softmax_cross_entropy(u2, _t) if flg else F.softmax(u2)
        # return F.mean_squared_error(self.policy_greedy(u3), _t) if flg else u3
        return F.mean_squared_error(u3, _t) if flg else u3
    
    def backpropagation(self, loss):
        loss.backward()
        self.optimizer.update()
    
    def init_grads(self):
        self.optimizer.zero_grads()
        
    def save_weight(self, model):
        serializers.save_npz(model, self.model)
        
    def load_weight(self, model):
        serializers.load_npz(model, self.model)
        
    def policy_greedy(self, actions):
        return np.max(actions.data, axis = 1)

In [ ]:
main()

[2017-05-12 22:20:22,093] Making new env: MountainCar-v0


1
2
